In [ ]:
from google.colab import files # this allows to upload data directly from your system

# upload files
uploaded = files.upload() 

Saving fakenewskdd2020.zip to fakenewskdd2020.zip


In [ ]:
# unzip the data/file(fakenewskdd2020)
!unzip /content/fakenewskdd2020.zip

Archive:  /content/fakenewskdd2020.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
# Load the data
import pandas as pd # it allows to do data analysis
import numpy as np
import matplotlib.pyplot as plt # it is relatively basic

# Load the train & test files
train = pd.read_csv("/content/train.csv",sep='\t',encoding='utf-8') # it reads the .csv file
test = pd.read_csv("/content/test.csv",sep='\t',encoding='utf-8')

In [ ]:
train.head() # allow you to visualise first n rows i.e 5 & it is Natural Language Process(NLP)

,text,label
0,Get the latest from TODAY Sign up for our news...,1
1,2d Conan On The Funeral Trump Will Be Invited...,1
2,It’s safe to say that Instagram Stories has fa...,0
3,Much like a certain Amazon goddess with a lass...,0
4,At a time when the perfect outfit is just one ...,0


** assumption of Bag of words : order does not **matter**

In [ ]:
import nltk # import ntlk---> Natural Language Toolkit allows u to work with Natural Language
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
train['text'].loc[0] # returns the very first data point/article

'Get the latest from TODAY Sign up for our newsletter  No one ever truly gets over losing a loved one, and Blake Shelton is no exception. He was just 14 when his older brother Richie died on Nov. 13, 1990. And, as Shelton noted in a tweet Monday, "It changed my life forever."  Richie was 24 when he died in a car accident in the Sheltons\' home state of Oklahoma. Two years ago, Shelton sent out a message for the 25th anniversary of his loss:  Richie, who was Blake\'s half-brother (they shared a mother), was a passenger in a car that collided with a school bus in Ada, south of Oklahoma City.  Richie, driver Redena McManus and a 3-year-old boy, Christopher McManus, all died during or shortly after the collision, while the bus driver and passengers were uninjured, according to police reports.  The accident has clearly remained with Blake, who told 60 Minutes in 2014, "I remember picking up the phone to call him a week after he was dead, to tell him something. I was picking up the phone to 

In [ ]:
train['label'].loc[0] # to know if news is real or fake we check its label if (label=0,true & label=1,fake) we check it as train['label'].loc[0]

'1'

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords # it removes unnessecary word like (is,am,are,and)
from nltk.stem import PorterStemmer # converting any word to its stem/root (running--->run) 
from sklearn.feature_extraction.text import TfidfVectorizer # it converts words to numbers as system can only understand numbers not words

# create a function to process the data using bag of words pipeline
def preprocess_data(data):

  # 1. Tokenization
  tk = RegexpTokenizer('\s+',gaps=True) # \s ---> space i.e. if space is available it tokenize the data 
  text_data = [] # List for storing the tokenized data
  for values in data.text:
    tokenized_data = tk.tokenize(values) # tokenize the news
    text_data.append(tokenized_data) # append the tokenized data to list

  # 2. Stopword Removal
  # Extract the stopwords
  sw = stopwords.words('english') # create a list of stopwords in english language
  clean_data = [] # List for storing the clean text  
  # remove the stopwords usin stopwords
  for data in text_data:# iterating over tokenized data
    clean_text = [words.lower() for words in data if words.lower() not in sw] # list comprehension
    clean_data.append(clean_text) # append the clean_text in the clean_data list

  # 3. Steaming
  # create a stemmer object
  ps = PorterStemmer()
  stemmed_data = [] # list for storing the stemmed data
  for data in clean_data:
    stemmed_text = [ps.stem(words) for words in data] # stem the words
    stemmed_data.append(stemmed_text) # Append the stemmed text  

  # flatten the stemmed data
  updated_data = []
  for data in stemmed_data:
    updated_data.append(" ".join(data))

  # TFID vector object
  tfidf = TfidfVectorizer()
  tfidf_matrix = tfidf.fit_transform(updated_data)

  return tfidf_matrix     

In [ ]:
# call the above function on the merged data
train_len = train.shape[0]
merged_data = pd.concat((train.drop('label',axis=1),test.drop('id',axis=1)),axis=0).reset_index().drop('index',axis=1)

In [ ]:
# preprocess the merged data
preprocessed_data = preprocess_data(merged_data)

In [ ]:
train_data = preprocessed_data[ : train_len]
test_data = preprocessed_data[train_len : ]

In [ ]:
# model selection 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data, train.label, test_size=0.2, random_state=42)

In [ ]:
# matrics
from sklearn.metrics import accuracy_score

# model
def compute_metrics(data,y_true,model_obj,model):

  # make predictions
  y_pred = model_obj.predict(data)
  # compute accuracy
  acc = accuracy_score(y_true = y_true,y_pred = y_pred)
  # make dataframe
  metrics = pd.DataFrame(data = np.array([acc]),index=[model],columns=['Accuracy Score'])
  return metrics

In [ ]:
# 1. LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

# model object
lr_reg = LogisticRegressionCV(Cs=20, cv=3, random_state=42)

# fit the model
lr_reg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


LogisticRegressionCV(Cs=20, class_weight=None, cv=3, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=None,
                     penalty='l2', random_state=42, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

In [ ]:
# compute the logistic regression metrics
lr_metrics = compute_metrics(X_test,y_test,lr_reg,'LogisticRegression')

In [ ]:
lr_metrics

,Accuracy Score
LogisticRegression,0.778557


In [ ]:
#2. naive bayes
from sklearn.naive_bayes import MultinomialNB

# model object
mnb = MultinomialNB(alpha=0.0)

# fit the object
mnb.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/naive_bayes.py:507: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


MultinomialNB(alpha=0.0, class_prior=None, fit_prior=True)

In [ ]:
# compute metrics 
mnb_metrics = compute_metrics(X_test,y_test,mnb,'Naive Bayes')
mnb_metrics

,Accuracy Score
Naive Bayes,0.719439


In [ ]:
#3. Decision tree
from sklearn.tree import DecisionTreeClassifier

# model object
df_clf = DecisionTreeClassifier()

# fit the object
df_clf.fit(X_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
dt_metrics = compute_metrics(X_test,y_test,df_clf,"DecisionTree")

In [ ]:
dt_metrics

,Accuracy Score
DecisionTree,0.692385


In [ ]:
from xgboost import XGBClassifier

# xgb model
xgb_model = XGBClassifier(n_estimators=200)
xgb_model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
xgb_metrics = compute_metrics(X_test,y_test,xgb_model,'XGBClassifier')

In [44]:
# concatenate all the metrics
model_metrics = pd.concat((lr_metrics,mnb_metrics,dt_metrics,xgb_metrics),axis=0).sort_values(by='Accuracy Score',ascending=False)

In [45]:
model_metrics

,Accuracy Score
LogisticRegression,0.778557
XGBClassifier,0.763527
Naive Bayes,0.719439
DecisionTree,0.692385


In [49]:
# make prediction---> xgbboost
predictions = xgb_model.predict(test_data)
# submission
test_ID = test.id
submissions = pd.DataFrame({'id':test_ID,'label':predictions})

In [50]:
submissions.head()

,id,label
0,2,0
1,3,0
2,4,1
3,5,1
4,6,0


In [51]:
submissions.to_csv("/content/predictions_4.csv",index=False)